In [11]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq
import csv
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Groq API and Models 
Groq_Token = "API_Key_here"

groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

In [13]:
og_filenames = ['A_Laying_S1.csv', 'A_Sitting_S2.csv', 'A_Standing_S3.csv', 'A_Walking_S4.csv', 'A_Downstairs_S5.csv', 'A_Upstairs_S1.csv']
normalized_filenames = ['Normalized_A_Laying_S1.csv', 'Normalized_A_Sitting_S2.csv', 'Normalized_A_Standing_S3.csv', 'Normalized_A_Walking_S4.csv', 'Normalized_A_Downstairs_S5.csv', 'Normalized_A_Upstairs_S1.csv']

for i in range(len(og_filenames)):
    og_data = og_filenames[i]
    df = pd.read_csv(og_data)

    # Normalizing the collected data between -1 and 1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    normalized_data = scaler.fit_transform(df)
    normalized_df = pd.DataFrame(normalized_data, columns=df.columns)
    output_file = normalized_filenames[i]
    normalized_df.to_csv(output_file, index=False)
    print(f"Normalized data saved to {output_file}")

Normalized data saved to Normalized_A_Laying_S1.csv
Normalized data saved to Normalized_A_Sitting_S2.csv
Normalized data saved to Normalized_A_Standing_S3.csv
Normalized data saved to Normalized_A_Walking_S4.csv
Normalized data saved to Normalized_A_Downstairs_S5.csv
Normalized data saved to Normalized_A_Upstairs_S1.csv


In [14]:
examples = f"""
accx = -0.1994058, accy = -0.03393813, accz = 0.9722126 : LAYING
accx = 0.9922325, accy = 0.0677996, accz = 0.2404148 : SITTING
accx = 0.9571659, accy = -0.1973788, accz = -0.3444446 : STANDING
accx = 1.073806, accy = -0.005115331, accz = 0.1358182 : WALKING
accx = 0.8900631, accy = 0.0321876, accz = -0.1827345 : WALKING_DOWNSTAIRS
accx = 1.078195, accy = -0.2588233, accz = -0.1880357 : WALKING_UPSTAIRS
"""

#using Arul's Data to test
test_samples = ['accx = 0.077531646, accy = 0.44577162, accz = 0.217744455',
                'accx = 0.441270896, accy = -0.453851917, accz = 0.6225188',
                'accx = 0.762343096, accy = 0.660353021, accz = -0.778451958',
                'accx = -0.723704261, accy = 0.165791089, accz = 0.069718595',
                'accx = -0.564597068, accy = 0.30039631, accz = 0.629232477',
                'accx = 0.360410855, accy = -0.106671447, accz = -0.150016108']
#LAYING, SITTING, STANDING, WALKING, WALKING_DOWNSTAIRS, WALKING_UPSTAIRS

query = f"""
* You are a activity recognition model. 
* Your task is to analyze the given data in form of accelerations in x, y and z directions and classify the activity as 'LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS' or 'WALKING_UPSTAIRS'. 
* Provide the activity label and, if necessary, a brief explanation of your reasoning.

A few examples are as follows:
{examples}

Analyze the following data and predict the activities:
1. {test_samples[0]}
2. {test_samples[1]}
3. {test_samples[2]}
4. {test_samples[3]}
5. {test_samples[4]}
6. {test_samples[5]}
""" 

#we use the same models we used for Task 3 to maintain consistency
model_name = "llama3.1-8b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("Answer from llama3.1-8b: ")
print(answer.content)

model_name = "llama3.1-70b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("\n Answer from llama3.1-70b: ")
print(answer.content)

Answer from llama3.1-8b: 
Based on the given data, I will analyze each sample and predict the activity.

1. accx = 0.077531646, accy = 0.44577162, accz = 0.217744455
The acceleration values in the x and y directions are relatively low, while the acceleration in the z direction is moderate. This suggests that the person is in a relatively stable position. Based on this, I predict the activity as: LAYING

2. accx = 0.441270896, accy = -0.453851917, accz = 0.6225188
The acceleration values in the x and y directions are moderate, while the acceleration in the z direction is high. This suggests that the person is moving in a way that involves a significant vertical component. Based on this, I predict the activity as: WALKING_UPSTAIRS

3. accx = 0.762343096, accy = 0.660353021, accz = -0.778451958
The acceleration values in the x and y directions are moderate to high, while the acceleration in the z direction is low. This suggests that the person is moving in a way that involves a significan

In [15]:
y_test = pd.DataFrame(['LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS'])

#llama3.1-8b
y_pred = pd.DataFrame(['LAYING', 'WALKING_UPSTAIRS', 'WALKING', 'SITTING', 'LAYING', 'WALKING_DOWNSTAIRS'])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'llama3.1-8b :')
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

#llama3.1-70b
y_pred = pd.DataFrame(['LAYING', 'SITTING', 'STANDING', 'LAYING', 'SITTING', 'WALKING_DOWNSTAIRS'])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'llama3.1-70b :')
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

llama3.1-8b :
Accuracy: 0.17
Precision: 0.08
Recall: 0.17
llama3.1-70b :
Accuracy: 0.50
Precision: 0.33
Recall: 0.50


In [16]:
examples = f"""
accx = -0.1994058, accy = -0.03393813, accz = 0.9722126 : LAYING
accx = -0.02925219, accy = 0.7027735, accz = 0.7284641 : LAYING
accx = 0.9922325, accy = 0.0677996, accz = 0.2404148 : SITTING
accx = 1.02311, accy = -0.02337259, accz = -0.06775652 : SITTING
accx = 0.9571659, accy = -0.1973788, accz = -0.3444446 : STANDING
accx = 1.012637, accy = -0.1900052, accz = 0.05822079 : STANDING
accx = 1.073806, accy = -0.005115331, accz = 0.1358182 : WALKING
accx = 1.415695, accy = 0.05032975, accz = -0.06400823 : WALKING
accx = 0.8900631, accy = 0.0321876, accz = -0.1827345 : WALKING_DOWNSTAIRS
accx = 1.304819, accy = -0.2678744, accz = 0.3007964 : WALKING_DOWNSTAIRS
accx = 1.078195, accy = -0.2588233, accz = -0.1880357 : WALKING_UPSTAIRS
accx = 1.479692, accy = -0.6064999, accz = 0.06833482 : WALKING_UPSTAIRS
"""

#using Arul's Data to test
test_samples = ['accx = 0.077531646, accy = 0.44577162, accz = 0.217744455',
                'accx = 0.441270896, accy = -0.453851917, accz = 0.6225188',
                'accx = 0.762343096, accy = 0.660353021, accz = -0.778451958',
                'accx = -0.723704261, accy = 0.165791089, accz = 0.069718595',
                'accx = -0.564597068, accy = 0.30039631, accz = 0.629232477',
                'accx = 0.360410855, accy = -0.106671447, accz = -0.150016108']
#LAYING, SITTING, STANDING, WALKING, WALKING_DOWNSTAIRS, WALKING_UPSTAIRS

query = f"""
* You are a activity recognition model. 
* Your task is to analyze the given data in form of accelerations in x, y and z directions and classify the activity as 'LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS' or 'WALKING_UPSTAIRS'. 
* Provide the activity label and, if necessary, a brief explanation of your reasoning.

A few examples are as follows:
{examples}

Analyze the following data and predict the activities:
1. {test_samples[0]}
2. {test_samples[1]}
3. {test_samples[2]}
4. {test_samples[3]}
5. {test_samples[4]}
6. {test_samples[5]}
""" 

#we use the same models we used for Task 3 to maintain consistency
model_name = "llama3.1-8b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("Answer from llama3.1-8b: ")
print(answer.content)

model_name = "llama3.1-70b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("\n Answer from llama3.1-70b: ")
print(answer.content)

Answer from llama3.1-8b: 
Based on the given data, I will analyze each sample and predict the activity.

1. accx = 0.077531646, accy = 0.44577162, accz = 0.217744455
The acceleration values in the x and y directions are relatively low, while the acceleration in the z direction is moderate. This suggests that the person is in a stable position, likely sitting. Therefore, the predicted activity is: **SITTING**

2. accx = 0.441270896, accy = -0.453851917, accz = 0.6225188
The acceleration values in the x and y directions are moderate, while the acceleration in the z direction is high. This suggests that the person is moving, likely walking. However, the acceleration values are not as high as those in the walking samples, and the z direction acceleration is not as dominant as in the walking samples. Therefore, the predicted activity is: **WALKING**

3. accx = 0.762343096, accy = 0.660353021, accz = -0.778451958
The acceleration values in the x and y directions are moderate, while the accel

In [17]:
y_test = pd.DataFrame(['LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS'])

#llama3.1-8b
y_pred = pd.DataFrame(['SITTING', 'WALKING', 'WALKING_DOWNSTAIRS', 'LAYING', 'LAYING', 'STANDING'])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'llama3.1-8b :')
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

#llama3.1-70b
y_pred = pd.DataFrame(['LAYING', 'LAYING', 'SITTING', 'LAYING', 'LAYING', 'STANDING'])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'llama3.1-70b :')
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

llama3.1-8b :
Accuracy: 0.00
Precision: 0.00
Recall: 0.00
llama3.1-70b :
Accuracy: 0.17
Precision: 0.04
Recall: 0.17
